In [1]:
import os, sys
import tensorflow as tf
from pixel2mesh.models import GCN
from pixel2mesh.fetcher import *
from pixel2mesh.cd_dist import nn_distance
sys.path.append('pixel2mesh/external')
from tf_approxmatch import approx_match, match_cost

flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('data_list', 'pixel2mesh/hitoshi_utils/test_list.txt', 'Data list path.')
flags.DEFINE_float('learning_rate', 3e-5, 'Initial learning rate.')
flags.DEFINE_integer('hidden', 192, 'Number of units in  hidden layer.')
flags.DEFINE_integer('feat_dim', 963, 'Number of units in perceptual feature layer.')
flags.DEFINE_integer('coord_dim', 3, 'Number of units in output layer.')
flags.DEFINE_float('weight_decay', 5e-6, 'Weight decay for L2 loss.')

num_blocks = 3
num_supports = 2
placeholders = {
    'features': tf.placeholder(tf.float32, shape=(None, 3)), # initial 3D coordinates
    'img_inp': tf.placeholder(tf.float32, shape=(224, 224, 3)), # input image to network
    'labels': tf.placeholder(tf.float32, shape=(None, 6)), # ground truth (point cloud with vertex normal)
    'support1': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)], # graph structure in the first block
    'support2': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)], # graph structure in the second block
    'support3': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)], # graph structure in the third block
    'faces': [tf.placeholder(tf.int32, shape=(None, 4)) for _ in range(num_blocks)], # helper for face loss (not used)
    'edges': [tf.placeholder(tf.int32, shape=(None, 2)) for _ in range(num_blocks)], # helper for normal loss
    'lape_idx': [tf.placeholder(tf.int32, shape=(None, 10)) for _ in range(num_blocks)], # helper for laplacian regularization
    'pool_idx': [tf.placeholder(tf.int32, shape=(None, 2)) for _ in range(num_blocks-1)] # helper for graph unpooling
}
model = GCN(placeholders, logging=True)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [2]:
# Construct feed dictionary
def construct_feed_dict(pkl, placeholders):
        coord = pkl[0]
        pool_idx = pkl[4]
        faces = pkl[5]
        lape_idx = pkl[7]
        edges = []
        for i in range(1,4):
                adj = pkl[i][1]
                edges.append(adj[0])
        feed_dict = dict()
        feed_dict.update({placeholders['features']: coord})
        feed_dict.update({placeholders['edges'][i]: edges[i] for i in range(len(edges))})
        feed_dict.update({placeholders['faces'][i]: faces[i] for i in range(len(faces))})
        feed_dict.update({placeholders['pool_idx'][i]: pool_idx[i] for i in range(len(pool_idx))})
        feed_dict.update({placeholders['lape_idx'][i]: lape_idx[i] for i in range(len(lape_idx))})
        feed_dict.update({placeholders['support1'][i]: pkl[1][i] for i in range(len(pkl[1]))})
        feed_dict.update({placeholders['support2'][i]: pkl[2][i] for i in range(len(pkl[2]))})
        feed_dict.update({placeholders['support3'][i]: pkl[3][i] for i in range(len(pkl[3]))})
        return feed_dict

def f_score(label, predict, dist_label, dist_pred, threshold):
        num_label = label.shape[0]
        num_predict = predict.shape[0]
        f_scores = []
        for i in range(len(threshold)):
                num = len(np.where(dist_label <= threshold[i])[0])
                recall = 100.0 * num / num_label
                num = len(np.where(dist_pred <= threshold[i])[0])
                precision = 100.0 * num / num_predict
                f_scores.append((2*precision*recall)/(precision+recall+1e-8))
        return np.array(f_scores)

In [4]:
# Load data
data = DataFetcher(FLAGS.data_list)
data.setDaemon(True) ####
data.start()
train_number = data.number

# Initialize session
# xyz1:dataset_points * 3, xyz2:query_points * 3
xyz1=tf.placeholder(tf.float32,shape=(None, 3))
xyz2=tf.placeholder(tf.float32,shape=(None, 3))
# chamfer distance
dist1,idx1,dist2,idx2 = nn_distance(xyz1, xyz2)
# earth mover distance, notice that emd_dist return the sum of all distance
match = approx_match(xyz1, xyz2)
emd_dist = match_cost(xyz1, xyz2, match)

config=tf.ConfigProto()
config.gpu_options.allow_growth=True
config.allow_soft_placement=True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())
model.load(sess)

# Construct feed dictionary
pkl = pickle.load(open('pixel2mesh/utils/ellipsoid/info_ellipsoid.dat', 'rb'))
feed_dict = construct_feed_dict(pkl, placeholders)

INFO:tensorflow:Restoring parameters from pixel2mesh/hitoshi_utils/hitoshi_checkpoint/gcn.ckpt
Model restored from file: pixel2mesh/hitoshi_utils/hitoshi_checkpoint/gcn.ckpt


In [ ]:
class_name = {'02828884':'bench','03001627':'chair','03636649':'lamp','03691459':'speaker','04090263':'firearm','04379243':'table','04530566':'watercraft','02691156':'plane','02933112':'cabinet','0295834\
3':'car','03211117':'monitor','04256520':'couch','04401088':'cellphone'}
model_number = {i:0 for i in class_name}
sum_f = {i:0 for i in class_name}
sum_cd = {i:0 for i in class_name}
sum_emd = {i:0 for i in class_name}

for iters in range(train_number):
        img_inp, label, model_id = data.fetch()
        feed_dict.update({placeholders['img_inp']: img_inp})
        feed_dict.update({placeholders['labels']: label})
        predict = sess.run(model.output3, feed_dict=feed_dict)
        label = label[:, :3]
        d1,i1,d2,i2,emd = sess.run([dist1,idx1,dist2,idx2, emd_dist], feed_dict={xyz1:label,xyz2:predict})
        cd = np.mean(d1) + np.mean(d2)
        class_id = model_id.split('_')[0]
        model_number[class_id] += 1.0
        sum_f[class_id] += f_score(label,predict,d1,d2,[0.0001, 0.0002])
        sum_cd[class_id] += cd # cd is the mean of all distance
        sum_emd[class_id] += emd[0] # emd is the sum of all distance
        print 'processed number', iters

log = open('record_evaluation.txt', 'a')
for item in model_number:
        number = model_number[item] + 1e-8
        f = sum_f[item] / number
        cd = (sum_cd[item] / number) * 1000 #cd is the mean of all distance, cd is L2
        emd = (sum_emd[item] / number) * 0.01 #emd is the sum of all distance, emd is L1
        print class_name[item], int(number), f, cd, emd
        print >> log, class_name[item], int(number), f, cd, emd
log.close()
sess.close()
data.shutdown()
print 'Testing Finished!'

processed number 0
processed number 1
processed number 2
processed number 3
processed number 4
processed number 5
processed number 6
processed number 7
processed number 8
processed number 9
processed number 10
processed number 11
processed number 12
processed number 13
processed number 14
processed number 15
processed number 16
processed number 17
processed number 18
processed number 19
processed number 20
processed number 21
processed number 22
processed number 23
processed number 24
processed number 25
processed number 26
processed number 27
processed number 28
processed number 29
processed number 30
processed number 31
processed number 32
processed number 33
processed number 34
processed number 35
processed number 36
processed number 37
processed number 38
processed number 39
processed number 40
processed number 41
processed number 42
processed number 43
processed number 44
processed number 45
processed number 46
processed number 47
processed number 48
processed number 49
processed 

processed number 396
processed number 397
processed number 398
processed number 399
processed number 400
processed number 401
processed number 402
processed number 403
processed number 404
processed number 405
processed number 406
processed number 407
processed number 408
processed number 409
processed number 410
processed number 411
processed number 412
processed number 413
processed number 414
processed number 415
processed number 416
processed number 417
processed number 418
processed number 419
processed number 420
processed number 421
processed number 422
processed number 423
processed number 424
processed number 425
processed number 426
processed number 427
processed number 428
processed number 429
processed number 430
processed number 431
processed number 432
processed number 433
processed number 434
processed number 435
processed number 436
processed number 437
processed number 438
processed number 439
processed number 440
processed number 441
processed number 442
processed num

processed number 787
processed number 788
processed number 789
processed number 790
processed number 791
processed number 792
processed number 793
processed number 794
processed number 795
processed number 796
processed number 797
processed number 798
processed number 799
processed number 800
processed number 801
processed number 802
processed number 803
processed number 804
processed number 805
processed number 806
processed number 807
processed number 808
processed number 809
processed number 810
processed number 811
processed number 812
processed number 813
processed number 814
processed number 815
processed number 816
processed number 817
processed number 818
processed number 819
processed number 820
processed number 821
processed number 822
processed number 823
processed number 824
processed number 825
processed number 826
processed number 827
processed number 828
processed number 829
processed number 830
processed number 831
processed number 832
processed number 833
processed num

processed number 1170
processed number 1171
processed number 1172
processed number 1173
processed number 1174
processed number 1175
processed number 1176
processed number 1177
processed number 1178
processed number 1179
processed number 1180
processed number 1181
processed number 1182
processed number 1183
processed number 1184
processed number 1185
processed number 1186
processed number 1187
processed number 1188
processed number 1189
processed number 1190
processed number 1191
processed number 1192
processed number 1193
processed number 1194
processed number 1195
processed number 1196
processed number 1197
processed number 1198
processed number 1199
processed number 1200
processed number 1201
processed number 1202
processed number 1203
processed number 1204
processed number 1205
processed number 1206
processed number 1207
processed number 1208
processed number 1209
processed number 1210
processed number 1211
processed number 1212
processed number 1213
processed number 1214
processed 

processed number 1544
processed number 1545
processed number 1546
processed number 1547
processed number 1548
processed number 1549
processed number 1550
processed number 1551
processed number 1552
processed number 1553
processed number 1554
processed number 1555
processed number 1556
processed number 1557
processed number 1558
processed number 1559
processed number 1560
processed number 1561
processed number 1562
processed number 1563
processed number 1564
processed number 1565
processed number 1566
processed number 1567
processed number 1568
processed number 1569
processed number 1570
processed number 1571
processed number 1572
processed number 1573
processed number 1574
processed number 1575
processed number 1576
processed number 1577
processed number 1578
processed number 1579
processed number 1580
processed number 1581
processed number 1582
processed number 1583
processed number 1584
processed number 1585
processed number 1586
processed number 1587
processed number 1588
processed 

...

processed number 45509
processed number 45510
processed number 45511
processed number 45512
processed number 45513
processed number 45514
processed number 45515
processed number 45516
processed number 45517
processed number 45518
processed number 45519
processed number 45520
processed number 45521
processed number 45522
processed number 45523
processed number 45524
processed number 45525
processed number 45526
processed number 45527
processed number 45528
processed number 45529
processed number 45530
processed number 45531
processed number 45532
processed number 45533
processed number 45534
processed number 45535
processed number 45536
processed number 45537
processed number 45538
processed number 45539
processed number 45540
processed number 45541
processed number 45542
processed number 45543
processed number 45544
processed number 45545
processed number 45546
processed number 45547
processed number 45548
processed number 45549
processed number 45550
processed number 45551
processed n

processed number 45866
processed number 45867
processed number 45868
processed number 45869
processed number 45870
processed number 45871
processed number 45872
processed number 45873
processed number 45874
processed number 45875
processed number 45876
processed number 45877
processed number 45878
processed number 45879
processed number 45880
processed number 45881
processed number 45882
processed number 45883
processed number 45884
processed number 45885
processed number 45886
processed number 45887
processed number 45888
processed number 45889
processed number 45890
processed number 45891
processed number 45892
processed number 45893
processed number 45894
processed number 45895
processed number 45896
processed number 45897
processed number 45898
processed number 45899
processed number 45900
processed number 45901
processed number 45902
processed number 45903
processed number 45904
processed number 45905
processed number 45906
processed number 45907
processed number 45908
processed n

processed number 46223
processed number 46224
processed number 46225
processed number 46226
processed number 46227
processed number 46228
processed number 46229
processed number 46230
processed number 46231
processed number 46232
processed number 46233
processed number 46234
processed number 46235
processed number 46236
processed number 46237
processed number 46238
processed number 46239
processed number 46240
processed number 46241
processed number 46242
processed number 46243
processed number 46244
processed number 46245
processed number 46246
processed number 46247
processed number 46248
processed number 46249
processed number 46250
processed number 46251
processed number 46252
processed number 46253
processed number 46254
processed number 46255
processed number 46256
processed number 46257
processed number 46258
processed number 46259
processed number 46260
processed number 46261
processed number 46262
processed number 46263
processed number 46264
processed number 46265
processed n

processed number 46580
processed number 46581
processed number 46582
processed number 46583
processed number 46584
processed number 46585
processed number 46586
processed number 46587
processed number 46588
processed number 46589
processed number 46590
processed number 46591
processed number 46592
processed number 46593
processed number 46594
processed number 46595
processed number 46596
processed number 46597
processed number 46598
processed number 46599
processed number 46600
processed number 46601
processed number 46602
processed number 46603
processed number 46604
processed number 46605
processed number 46606
processed number 46607
processed number 46608
processed number 46609
processed number 46610
processed number 46611
processed number 46612
processed number 46613
processed number 46614
processed number 46615
processed number 46616
processed number 46617
processed number 46618
processed number 46619
processed number 46620
processed number 46621
processed number 46622
processed n

processed number 46938
processed number 46939
processed number 46940
processed number 46941
processed number 46942
processed number 46943
processed number 46944
processed number 46945
processed number 46946
processed number 46947
processed number 46948
processed number 46949
processed number 46950
processed number 46951
processed number 46952
processed number 46953
processed number 46954
processed number 46955
processed number 46956
processed number 46957
processed number 46958
processed number 46959
processed number 46960
processed number 46961
processed number 46962
processed number 46963
processed number 46964
processed number 46965
processed number 46966
processed number 46967
processed number 46968
processed number 46969
processed number 46970
processed number 46971
processed number 46972
processed number 46973
processed number 46974
processed number 46975
processed number 46976
processed number 46977
processed number 46978
processed number 46979
processed number 46980
processed n